# Trabajo Practico 6:


In [98]:
import xml.etree.ElementTree as ET
import nltk
from nltk.collocations import *
import os
import math
import itertools
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import stopwords
import scipy as sp
import numpy as np

In [2]:
tree = ET.parse('ap.xml')
root = tree.getroot()

corpusDic = {}
totalString = ""
for doc in root:
    docno, text = doc
    docnum = (docno.text).replace(" ", "")
    corpusDic[docnum] = text.text
    
    textWithAddedSpace = text.text + " "
    totalString += textWithAddedSpace

## 1.1) Levantar el corpus AP, separando cada noticia como un elemento distinto
en un diccionario (<DOCNO> : <TEXT>).

In [3]:
vocabulario = {}
stringsNoDeseados = [',', '.', "''", "'", "!",';',':', '``', "__", '@', '|', '...', '(', ')', '{', '}']
for key in corpusDic.keys():
    tokens = nltk.word_tokenize(corpusDic[key])
    
    for word in tokens:
        if word not in stringsNoDeseados:
            if word in vocabulario:
                vocabulario[word] += 1
            else:
                vocabulario[word] = 1
                

In [17]:
vtokens = nltk.word_tokenize(totalString)
for i in range(10):
    print(vtokens[i])

A
16-year-old
student
at
a
private
Baptist
school
who
allegedly


## 1.2) Calcular el tamaño del vocabulario. 

In [4]:
print("tamaño del vocabulario: ", len(vocabulario.keys()))

tamaño del vocabulario:  47454


## 1.3) Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [5]:
mostRepeated500 = [word for word in sorted(vocabulario, key=vocabulario.get, reverse=True)][0:501]

In [18]:
bigram_measures_class = nltk.collocations.BigramAssocMeasures

#esto hace los bigramas de todas las palabras en el texto
finder = nltk.collocations.BigramCollocationFinder.from_words(vtokens,window_size=5)


In [19]:
sorted(finder.nbest(bigram_measures_class.pmi, 30))
#finder.score_ngram(bigram_measures_class.pmi, '3', '/') 

[("'LL", 'WALK'),
 ("'Til", 'Becomes'),
 ("'Til", 'Tear'),
 ('-547,000', 'sundry'),
 ('-Bgistratwissmou', 'criminalp`ummo'),
 ('-Los', 'Angeles-Tokyo'),
 ('-More', 'armor-piercing'),
 ('-Patrick', 'Kieran'),
 ('.04', '158.89'),
 ('.07', '.13'),
 ('.09', '361.08'),
 ('.11', '193.44'),
 ('.26', '195.59'),
 ('.36', '147.51'),
 ('.41', '304.54'),
 ('.43', '270.16'),
 ('.44', 'Magnums'),
 ('.51', '296.06'),
 ('.61', '66.18'),
 ('.62', '156.79'),
 ('.63', '298.34'),
 ('.64', '310.11'),
 ('.67', '75.24'),
 ('.68', '1.58'),
 ('.79', '355.54'),
 ('.89', '2,132.11'),
 ('.93', '184.04'),
 ('0.01', '155.36'),
 ('0.02', '1,878.9'),
 ('0.027', '0.109')]

In [30]:
#para mi esto esta mal xq te hace los bigramas entre las palabras de mosrRepeated500.

scores = {}
for word_1, word_2 in itertools.combinations(mostRepeated500, 2):
    score = finder.score_ngram(bigram_measures_class.pmi, word_1, word_2) 
    # FALTA HACER EL APPEND!! EL IF ES EL PROBLEMA!!
    scores[(word_1, word_2)] = score if score is not None else float('-inf')

aux = sorted(scores, key=scores.get, reverse=True)
[(au, scores[au]) for au in aux[:10]]

[(('South', 'Africa'), 8.652361975378483),
 (('interest', 'rates'), 8.532893251786337),
 (('news', 'conference'), 8.244661043385566),
 (('New', 'York'), 8.062812652158431),
 (('United', 'States'), 8.06188036051849),
 (('West', 'German'), 7.983531966464916),
 (('East', 'Germany'), 7.762227820171429),
 (('West', 'Germany'), 7.738697068761287),
 (('Soviet', 'Union'), 7.614216942440752),
 (('told', 'reporters'), 7.557934639238603)]

## 2.1) Procesar el texto, tokenizando eliminando signos de puntuación.


In [71]:
texto = open("Darwin.txt","r").read()
tokenizer = nltk.tokenize.TreebankWordTokenizer()
tokens = nltk.word_tokenize(texto)
stopset = set(stopwords.words('english'))

tokens = [w for w in tokens if (not w in stopset and not w in stringsNoDeseados)]

## 2.2) Siguiendo el artı́culo de la sección, calcular la autocorrelación para estimar la distribución de la palabra a lo largo del texto.


In [ ]:
dicGausXPalabra = {}
palabras_buscar = ['plants','instinct','for']
for indice in range(len(tokens)):
    if tokens[indice] in palabras_buscar:
        if tokens[indice] in dicGausXPalabra:
            dicGausXPalabra[tokens[indice]].append(sp.stats.norm(indice,50))
        else:
            dicGausXPalabra[tokens[indice]] = [sp.stats.norm(indice,50)]

In [ ]:
dicSumGausXPalabra = {}
for key, gausList in dicGausXPalabra.items():
    print(key)
    for index in range(len(tokens)):
        sumGausianas = 0
        for gausiana in dicGausXPalabra[key]:
            sumGausianas += gausiana.pdf(indice)
        dicSumGausXPalabra[key] = sumGausianas
print(dicSumGausXPalabra)

## 2.3)

In [93]:
len(tokens)

86297

## 2.4)